In [1]:
import pandas as pd
import numpy as np
import gc
import datetime
import glob
import glob

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 8000)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None  # default='warn'
np.set_printoptions(suppress=False)

In [3]:
l = [pd.read_csv(filename, low_memory=False) for filename in glob.glob(r"C:\Users\****\Downloads\chain/*.txt")]
df = pd.concat(l, axis=0)

In [4]:
def seperate(data):
    data = data.rename(columns=lambda x: x.strip())
    # data.columns = [c.replace("]", "") for c in list(data.columns)]
    # data.columns = [c.replace("[", "") for c in list(data.columns)]
    data['[QUOTE_DATE]'] = pd.to_datetime(data['[QUOTE_DATE]'], format='%Y-%m-%d')
    data['[EXPIRE_DATE]'] = pd.to_datetime(data['[EXPIRE_DATE]'], format='%Y-%m-%d')
    data['[QUOTE_READTIME]'] = pd.to_datetime(data['[QUOTE_READTIME]'], format='%Y-%m-%d %H:%M')
   
    dfcall = data[['[QUOTE_UNIXTIME]', '[QUOTE_READTIME]', '[QUOTE_DATE]',
                   '[QUOTE_TIME_HOURS]','[UNDERLYING_LAST]', '[EXPIRE_DATE]','[EXPIRE_UNIX]', '[DTE]', '[STRIKE]','[STRIKE_DISTANCE]','[STRIKE_DISTANCE_PCT]',
                   '[C_IV]','[C_DELTA]','[C_GAMMA]','[C_VEGA]','[C_THETA]','[C_RHO]','[C_VOLUME]', '[C_LAST]', '[C_SIZE]','[C_BID]', '[C_ASK]']]
    dfcall['optype'] = 'C'
    dfcall.rename(columns={'[C_IV]': 'iv', '[C_DELTA]': 'delta', '[C_GAMMA]': 'gamma', '[C_VEGA]': 'vega', '[C_THETA]':'theta','[C_RHO]':'rho','[C_VOLUME]':'vol','[C_LAST]':'last','[C_SIZE]':'size','[C_BID]':'bid','[C_ASK]':'ask'}, inplace = True)
                 
                 
    dfput = data[['[QUOTE_UNIXTIME]', '[QUOTE_READTIME]','[QUOTE_DATE]','[QUOTE_TIME_HOURS]','[UNDERLYING_LAST]', '[EXPIRE_DATE]','[EXPIRE_UNIX]', '[DTE]', '[STRIKE]','[STRIKE_DISTANCE]','[STRIKE_DISTANCE_PCT]',
                    '[P_IV]','[P_DELTA]','[P_GAMMA]','[P_VEGA]','[P_THETA]','[P_RHO]','[P_VOLUME]','[P_LAST]', '[P_SIZE]','[P_BID]','[P_ASK]']]
   
    dfput.rename(columns={'[P_IV]': 'iv', '[P_DELTA]': 'delta', '[P_GAMMA]': 'gamma', '[P_VEGA]': 'vega', '[P_THETA]':'theta','[P_RHO]':'rho','[P_VOLUME]':'vol','[P_LAST]':'last','[P_SIZE]':'size','[P_BID]':'bid','[P_ASK]':'ask'}, inplace = True)

    dfput['optype'] = 'P'
    dfchain = pd.concat([dfcall, dfput], axis=0)
    # UNIXTIME To HKT
    dfchain['[QUOTE_UNIXTIME]'] = pd.to_datetime(dfchain['[QUOTE_UNIXTIME]'], unit='s',utc=True).dt.tz_convert('HongKong')
    dfchain['[QUOTE_UNIXTIME]'] = dfchain['[QUOTE_UNIXTIME]'].dt.tz_localize(None)
    # Expire_UNIX
    dfchain['[EXPIRE_UNIX]'] = pd.to_datetime(dfchain['[EXPIRE_UNIX]'], unit='s',utc=True).dt.tz_convert('HongKong')
    dfchain['[EXPIRE_UNIX]'] = dfchain['[EXPIRE_UNIX]'].dt.tz_localize(None)
    dfchain.drop(columns={'[QUOTE_TIME_HOURS]','[QUOTE_DATE]'}, inplace = True)
    dfchain['[DTE]'] = dfchain['[DTE]'].astype(np.int16)
    dfchain['[STRIKE]'] = dfchain['[STRIKE]'].astype(np.int32)
    dfchain.columns = [c.replace("]", "") for c in list(dfchain.columns)]
    dfchain.columns = [c.replace("[", "") for c in list(dfchain.columns)]
    dfchain.rename(columns={'QUOTE_UNIXTIME': 'Quote_Date_HKT', '[QUOTE_READTIME]': 'Date_US',
                           'EXPIRE_UNIX': 'Expire_Date_HKT','EXPIRE_DATE':'Expire_Date_US',
                           'QUOTE_READTIME':'Quote_Date_US','UNDERLYING_LAST':'SPX_Last','STRIKE':'Strike','STRIKE_DISTANCE':'Distance','STRIKE_DISTANCE_PCT':'Distance_Pct'}, inplace = True)
    dfchain['iv'] = pd.to_numeric(dfchain['iv'], errors = 'coerce')
    dfchain['delta'] = pd.to_numeric(dfchain['delta'], errors = 'coerce')
    dfchain['gamma'] = pd.to_numeric(dfchain['gamma'], errors = 'coerce')
    dfchain['vega'] = pd.to_numeric(dfchain['vega'], errors = 'coerce')
    dfchain['vega'] = pd.to_numeric(dfchain['vega'], errors = 'coerce')
    dfchain['theta'] = pd.to_numeric(dfchain['theta'], errors = 'coerce')
    dfchain['rho'] = pd.to_numeric(dfchain['rho'], errors = 'coerce')
    dfchain['vol'] = pd.to_numeric(dfchain['vol'], errors = 'coerce')
    dfchain['last'] = pd.to_numeric(dfchain['last'], errors = 'coerce').astype(np.float16)
    dfchain['bid'] = pd.to_numeric(dfchain['bid'], errors = 'coerce').astype(np.float16)
    dfchain['ask'] = pd.to_numeric(dfchain['ask'], errors = 'coerce').astype(np.float16)
    dfchain['Distance'] = dfchain['Distance'].astype(np.float16)
    dfchain['Distance_Pct'] = dfchain['Distance_Pct'].astype(np.float16)
    dfchain['Contract_ID'] = "SPX"+dfchain['Expire_Date_US'].dt.strftime('%Y%m%d')+dfchain['optype']+dfchain['Strike'].astype(str)
    dfchain = dfchain[['Contract_ID','Quote_Date_HKT','Expire_Date_HKT','Quote_Date_US','Expire_Date_US','SPX_Last','optype','Strike','DTE','Distance', 'Distance_Pct', 'iv', 'delta', 'gamma', 'vega', 'theta', 'rho', 'vol', 'last', 'size', 'bid', 'ask']]
    dfchain  = dfchain.sort_values(['Quote_Date_HKT','DTE','optype','Strike'],ascending=[True,True,True,True])
    # df = df[df['[STRIKE_DISTANCE]']<2500]    
    # dfchain.drop(columns={'[QUOTE_UNIXTIME]','[QUOTE_TIME_HOURS]','[EXPIRE_UNIX]'}, inplace = True)
    return dfchain

In [5]:
df = seperate(df)

In [6]:
df.head()

,Contract_ID,Quote_Date_HKT,Expire_Date_HKT,Quote_Date_US,Expire_Date_US,SPX_Last,optype,Strike,DTE,Distance,Distance_Pct,iv,delta,gamma,vega,theta,rho,vol,last,size,bid,ask
0,SPX20220103C1200,2022-01-04 05:00:00,2022-01-04 05:00:00,2022-01-03 16:00:00,2022-01-03,4795.57,C,1200,0,3596.00,0.75,NaN,1.00,0.00,0.00,0.00,0.00,NaN,0.00,10 x 1,3582.00,3604.00
1,SPX20220103C1400,2022-01-04 05:00:00,2022-01-04 05:00:00,2022-01-03 16:00:00,2022-01-03,4795.57,C,1400,0,3396.00,0.71,NaN,1.00,0.00,0.00,0.00,0.00,NaN,0.00,10 x 1,3382.00,3404.00
2,SPX20220103C1600,2022-01-04 05:00:00,2022-01-04 05:00:00,2022-01-03 16:00:00,2022-01-03,4795.57,C,1600,0,3196.00,0.67,NaN,1.00,0.00,0.00,0.00,0.00,NaN,0.00,10 x 1,3182.00,3204.00
3,SPX20220103C1800,2022-01-04 05:00:00,2022-01-04 05:00:00,2022-01-03 16:00:00,2022-01-03,4795.57,C,1800,0,2996.00,0.62,NaN,1.00,0.00,0.00,0.00,0.00,NaN,0.00,10 x 1,2982.00,3004.00
4,SPX20220103C1900,2022-01-04 05:00:00,2022-01-04 05:00:00,2022-01-03 16:00:00,2022-01-03,4795.57,C,1900,0,2896.00,0.60,NaN,1.00,0.00,0.00,0.00,0.00,NaN,0.00,10 x 1,2882.00,2904.00
